In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv('ddos_dataset.csv')

Data cleaning


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Flow ID            500000 non-null  object 
 1   Timestamp          500000 non-null  object 
 2   Fwd Pkt Len Mean   500000 non-null  float64
 3   Fwd Seg Size Avg   500000 non-null  float64
 4   Init Fwd Win Byts  500000 non-null  int64  
 5   Init Bwd Win Byts  500000 non-null  int64  
 6   Fwd Seg Size Min   500000 non-null  int64  
 7   Label              500000 non-null  object 
dtypes: float64(2), int64(3), object(3)
memory usage: 30.5+ MB


In [5]:
df.head()

,Flow ID,Timestamp,Fwd Pkt Len Mean,Fwd Seg Size Avg,Init Fwd Win Byts,Init Bwd Win Byts,Fwd Seg Size Min,Label
0,172.31.69.28-18.216.200.189-80-52169-6,22/02/2018 12:27:57 AM,233.750000,233.750000,-1,32768,0,ddos
1,172.31.69.25-18.219.193.20-80-44588-6,16/02/2018 11:18:14 PM,0.000000,0.000000,-1,225,0,ddos
2,172.31.69.25-18.219.193.20-80-43832-6,16/02/2018 11:23:20 PM,114.333333,114.333333,-1,219,0,ddos
3,172.31.69.25-18.219.193.20-80-53346-6,16/02/2018 11:22:41 PM,233.750000,233.750000,-1,211,0,ddos
4,172.31.69.28-18.218.55.126-80-57856-6,21/02/2018 11:49:25 PM,233.750000,233.750000,-1,32768,0,ddos


In [ ]:
#Checking for null values
df.isnull().sum()

Flow ID              0
Timestamp            0
Fwd Pkt Len Mean     0
Fwd Seg Size Avg     0
Init Fwd Win Byts    0
Init Bwd Win Byts    0
Fwd Seg Size Min     0
Label                0
dtype: int64

In [6]:
for element in df:
  print(df[element].unique())


['172.31.69.28-18.216.200.189-80-52169-6'
 '172.31.69.25-18.219.193.20-80-44588-6'
 '172.31.69.25-18.219.193.20-80-43832-6' ...
 '151.101.34.109-172.31.64.29-443-53701-6'
 '172.31.0.2-172.31.65.44-53-60384-17'
 '192.168.10.3-192.168.10.17-53-15546-17']
['22/02/2018 12:27:57 AM' '16/02/2018 11:18:14 PM'
 '16/02/2018 11:23:20 PM' ... '03/07/2017 11:01:28 PM'
 '03/07/2017 07:45:54 PM' '04/07/2017 12:45:03 AM']
[233.75         0.         114.33333333 ... 107.72413793  91.91666667
  67.88235294]
[233.75         0.         114.33333333 ... 107.72413793  91.91666667
  67.88235294]
[   -1  8192  2049 ... 31760 64226 15525]
[32768   225   219 ... 46729 32682  4734]
[ 0 20  8 28 40 32 24 44 36]
['ddos' 'Benign']


In [7]:
from sklearn.preprocessing import LabelEncoder
encodings_dictionary = dict()
for c in df.columns:
    if df[c].dtype == "object":
        encodings_dictionary[c] = LabelEncoder()
        df[c] = encodings_dictionary[c].fit_transform(df[c])

In [8]:
df.head(5)

,Flow ID,Timestamp,Fwd Pkt Len Mean,Fwd Seg Size Avg,Init Fwd Win Byts,Init Bwd Win Byts,Fwd Seg Size Min,Label
0,382320,50948,233.750000,233.750000,-1,32768,0,1
1,369106,14079,0.000000,0.000000,-1,225,0,1
2,368742,14385,114.333333,114.333333,-1,219,0,1
3,373370,14346,233.750000,233.750000,-1,211,0,1
4,398779,48636,233.750000,233.750000,-1,32768,0,1


In [9]:
for element in df:
  print(df[element].unique())


[382320 369106 368742 ...  10774  92029 429071]
[50948 14079 14385 ...  8628  3845 11277]
[233.75         0.         114.33333333 ... 107.72413793  91.91666667
  67.88235294]
[233.75         0.         114.33333333 ... 107.72413793  91.91666667
  67.88235294]
[   -1  8192  2049 ... 31760 64226 15525]
[32768   225   219 ... 46729 32682  4734]
[ 0 20  8 28 40 32 24 44 36]
[1 0]


In [10]:
#Checking columns for redundancy
L1=[]
L2=[]
for value in df['Fwd Pkt Len Mean']:
  L1.append(value)
for value in df['Fwd Seg Size Avg']:
  L2.append(value)
if L1==L2:
  print('da')
else:
  print('ne')

ne


In [12]:
#Dropping useless columns
df = df.sort_values('Timestamp')
df.drop('Timestamp', axis=1,inplace=True)
df.drop('Flow ID', axis=1,inplace=True)


KeyError: ignored

Preprocessing

In [13]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

X = df.drop('Label',axis=1)
y= df['Label']

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)



Trying out models on the cleaned data

In [ ]:
from sklearn.svm import SVC

svm=SVC()
svm.fit(X_train, y_train)
predictions = svm.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,predictions))
print(confusion_matrix(y_test,predictions))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     80187
           1       0.99      0.99      0.99     19813

    accuracy                           1.00    100000
   macro avg       1.00      1.00      1.00    100000
weighted avg       1.00      1.00      1.00    100000

[[80027   160]
 [  133 19680]]


In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train,y_train)
lr_predictions = lr.predict(X_test)
print(classification_report(y_test,lr_predictions))
print(confusion_matrix(y_test,lr_predictions))

              precision    recall  f1-score   support

           0       0.96      0.94      0.95     80103
           1       0.77      0.83      0.80     19897

    accuracy                           0.92    100000
   macro avg       0.87      0.89      0.88    100000
weighted avg       0.92      0.92      0.92    100000

[[75284  4819]
 [ 3304 16593]]


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
knn_predictions = knn.predict(X_test)
print(classification_report(y_test,knn_predictions))
print(confusion_matrix(y_test,knn_predictions))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     80187
           1       1.00      1.00      1.00     19813

    accuracy                           1.00    100000
   macro avg       1.00      1.00      1.00    100000
weighted avg       1.00      1.00      1.00    100000

[[80170    17]
 [    6 19807]]


In [ ]:
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestClassifier
m = RFECV(RandomForestClassifier(), scoring='accuracy',n_jobs=50)
m.fit(X_train, y_train)
m_predictions = m.predict(X_test)
print(classification_report(y_test,m_predictions))
print(confusion_matrix(y_test,m_predictions))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00     80187
           1       1.00      1.00      1.00     19813

    accuracy                           1.00    100000
   macro avg       1.00      1.00      1.00    100000
weighted avg       1.00      1.00      1.00    100000

[[80185     2]
 [    4 19809]]


In [ ]:
rf = RandomForestClassifier(n_estimators=50)
rf.fit(X_train, y_train)
rf_predictions = rf.predict(X_test)
print(classification_report(y_test,rf_predictions))
print(confusion_matrix(y_test,rf_predictions))
rf.score(X_train, y_train)
rf.score(X_test, y_test)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     80171
           1       1.00      1.00      1.00     19829

    accuracy                           1.00    100000
   macro avg       1.00      1.00      1.00    100000
weighted avg       1.00      1.00      1.00    100000

[[80171     0]
 [    1 19828]]


0.99999

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dfm = DecisionTreeClassifier()
dfm.fit(X_train, y_train)
dfm_predictions = dfm.predict(X_test)
print(classification_report(y_test,dfm_predictions))
print(confusion_matrix(y_test,dfm_predictions))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     80103
           1       1.00      1.00      1.00     19897

    accuracy                           1.00    100000
   macro avg       1.00      1.00      1.00    100000
weighted avg       1.00      1.00      1.00    100000

[[80099     4]
 [    4 19893]]


In [ ]:
df.head()

,Fwd Pkt Len Mean,Fwd Seg Size Avg,Init Fwd Win Byts,Init Bwd Win Byts,Fwd Seg Size Min,Label
495277,0.0,0.0,-1,329,0,0
248956,81.0,81.0,-1,-1,0,0
112866,32.0,32.0,-1,-1,0,0
120912,75.0,75.0,-1,2053,0,0
216490,0.0,0.0,-1,0,0,0
